## Import necessary libraries

In [ ]:
import tifffile

from skimage.io import imread
from microfilm.microplot import microshow
from skimage import filters, morphology, measure
from matplotlib import pyplot as plt
from skimage.color import label2rgb

## Pre-processing Steps

### 1) Upload the image and analysis basics

In [ ]:
## Read Image and store it in a variable
full_image = imread("01-A-01.ome.tif")
## Visualize the numpy array
type(full_image)
full_image
## Check the image dimensions
full_image.shape

### 2) Visualize the full image

In [ ]:
microshow(full_image[1])

### 3) Visualize the full image plus the single channels

In [ ]:
nuclei = full_image[0]
cyto = full_image[1]

fig, axs = plt.subplots(1,3, figsize=(20,20))
microshow(full_image, ax=axs[0], label_text= 'Original', label_font_size=10, label_color='white')
microshow(nuclei, ax=axs[1], label_text='Nuclei', label_font_size=10, label_color='white')
microshow(cyto, ax=axs[2], label_text='Cytoplasm', label_font_size=10, label_color='white')

## Processing

### 1) Background removal with gaussian filter

In [ ]:
background_nuclei = filters.gaussian(nuclei, sigma=100, preserve_range=True)
denoised_nuclei = nuclei - background_nuclei
fig, axs = plt.subplots(1, 2, figsize=(10,10))

microshow(nuclei, ax=axs[0], label_text='Nuclei - Original', label_color='white')
microshow(denoised_nuclei, ax=axs[1], label_text='Nuclei - Processed', label_color='white')

### 2) Denoising 

In [ ]:
nuclei_gaussian = filters.gaussian(denoised_nuclei, sigma=2)

fig, axs = plt.subplots(1,2, figsize=(10,10))
microshow(nuclei, ax=axs[0], label_text='Nuclei - Original', label_color='white')
microshow(nuclei_gaussian, ax=axs[1], label_text='Nuclei - Processed', label_color='white')

### 3) Threshold, binarization and labelling

In [ ]:
# create a square with width and height = 3
square = morphology.square(5) 
square

threshold = filters.threshold_otsu(nuclei_gaussian)
binary_image = nuclei_gaussian >= threshold
opened = morphology.binary_opening(binary_image, square)
labeled_blobs = measure.label(opened)
cleaned = morphology.remove_small_objects(labeled_blobs, min_size=50)

fig, axs = plt.subplots(1,3, figsize=(10,10))

microshow(nuclei_gaussian, ax=axs[0], label_text='Original', label_color='white')
microshow(opened, ax=axs[1], label_text='Binary', label_color='white')
microshow(cleaned, ax=axs[2], label_text='Labels', label_color='white')

### 4) Overlap labelling on the original image

In [ ]:
image_label_overlay = label2rgb(cleaned, image=nuclei, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

## Saving result

In [ ]:
tifffile.imwrite('labelled.tif_01-A-01.ome.tif', image_label_overlay, 
                 dtype=image_label_overlay.dtype, 
                 shape=image_label_overlay.shape)